<a href="https://colab.research.google.com/github/sangitaGIT011/Myproj/blob/main/ETLprocessInExl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
#!pip install xlsxwriter
import os
import requests
import pandas as pd
from sqlalchemy import create_engine  # (not used in this example, but kept if needed later)

# Extract Data from URL

def extract() -> dict:

    url = 'http://universities.hipolabs.com/search?country=India'
    data = requests.get(url).json()
    return data

# Transform Data

def transform(data: dict) -> pd.DataFrame:
    df = pd.DataFrame(data)

    print(f"Total number of universities from API: {len(df)}")

    # Filter universities that have "Gujarat" in their name (case-insensitive)
    df = df[df["name"].str.contains("Gujarat", case=False, na=False)]

    print(f"Total number of universities in Gujarat: {len(df)}")

    # Convert list columns to comma-separated strings
    df["domains"] = [','.join(map(str, l)) for l in df["domains"]]
    df["web_pages"] = [','.join(map(str, l)) for l in df["web_pages"]]

    df = df.reset_index(drop=True)

    return df[["name", "domains", "web_pages", "state-province"]]

# Load data into Excel with formatting


def load(df: pd.DataFrame, filename: str = "gujarat_universities.xlsx") -> None:
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        df.to_excel(writer, index=False, sheet_name='Gujarat Universities')

        workbook = writer.book
        worksheet = writer.sheets['Gujarat Universities']

        # Set column widths for better readability
        worksheet.set_column('A:A', 40)  # name
        worksheet.set_column('B:B', 25)  # domains
        worksheet.set_column('C:C', 50)  # web_pages
        worksheet.set_column('D:D', 20)  # state-province

        print(f"Data successfully written to {filename}")

# Main ETL pipeline execution
if __name__ == "__main__":
    raw_data = extract()
    transformed_df = transform(raw_data)
    load(transformed_df)

    # Show full path of saved Excel file
    filename = "gujarat_universities.xlsx"
    filepath = os.path.abspath(filename)
    print(f"Full path of the Excel file: {filepath}")

    # Read the Excel file and display in console with improved formatting
    df = pd.read_excel(filename)

    # Better console display: no truncation and wider display width
    with pd.option_context('display.max_colwidth', None, 'display.width', 200):
        print(df.head())


Total number of universities from API: 473
Total number of universities in Gujarat: 5
Data successfully written to gujarat_universities.xlsx
Full path of the Excel file: /content/gujarat_universities.xlsx
                                         name                   domains                             web_pages state-province
0                  Gujarat Ayurved University     ayurveduniversity.com     http://www.ayurveduniversity.com/        Gujarat
1  Gujarat Technological University Ahmedabad                 gtu.ac.in                 http://www.gtu.ac.in/        Gujarat
2                Gujarat University Ahmedabad  gujaratuniversity.org.in  http://www.gujaratuniversity.org.in/        Gujarat
3   Hemchandracharay North Gujarat University                 ngu.ac.in                 http://www.ngu.ac.in/        Gujarat
4                    South Gujarat University              sgu.ernet.in              http://www.sgu.ernet.in/            NaN
